尝试把BPGD复现，在BP迭代之后加入decimation

In [94]:
from ldpc.codes import rep_code
from bposd.hgp import hgp
import time
import numpy as np

In [95]:
"""初始化表面码"""

h = rep_code(3)
surface_code = hgp(h1=h, h2=h, compute_distance=True)
H = surface_code.hz
n = surface_code.N

"""初始化decoder"""
from mqt.qecc import *  # UFDecoder
from ldpc import bp_decoder, bposd_decoder

p = 0.05  # 错误率

# our Decoder
T = 10
bp_decoder = bp_decoder(
    surface_code.hz,
    error_rate=None,
    channel_probs=[None],
    max_iter=T,
    bp_method=2,
    ms_scaling_factor=0,
)

# BPOSD Decoder
bposd_decoder = bposd_decoder(
    surface_code.hz,
    error_rate=p,
    channel_probs=[None],
    max_iter=T,
    bp_method="ms",
    ms_scaling_factor=0,
    osd_method="osd_0",
    osd_order=7,
)

# UFDecoder
code = Code(surface_code.hx, surface_code.hz)
uf_decoder = UFHeuristic()
uf_decoder.set_code(code)

我们方法预解码后的一些结果（先验）

In [96]:
B = np.array(
    [
        [1, 0, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 1, 0, 0],
        [0, 1, 0, 1, 0, 1, 0],
        [0, 1, 0, 0, 1, 0, 1],
        [0, 0, 1, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 0, 1],
    ]
)

"""初始化变量节点的对数似然比（log prob ratio）"""
W = [np.log((1 - p) / p) for _ in range(n)]
W_f = W[: H.shape[0]]
W_g = W[H.shape[0] :]

效果比较差，可能是接口写错了，还需要继续改（2025.2.14）

In [97]:
np.random.seed(1)

LFN = 25  # large fixed number

num_trials = 10000

our_num_success = 0
bposd_num_success = 0
uf_num_success = 0

for i in range(num_trials):

    """Generate random error"""
    error = np.zeros(surface_code.N).astype(int)
    for q in range(surface_code.N):
        if np.random.rand() < p:
            error[q] = 1

    """Obtain syndrome"""
    syndrome = surface_code.hz @ error % 2

    """Decode"""

    # BP+OSD Decoder
    bposd_decoder.decode(syndrome)
    bposd_residual_error = (bposd_decoder.osdw_decoding + error) % 2
    bposd_flag = (surface_code.lz @ bposd_residual_error % 2).any()
    if bposd_flag == 0:
        bposd_num_success += 1

    # UF Decoder
    uf_decoder.decode(syndrome)
    uf_result = np.array(uf_decoder.result.estimate).astype(int)
    uf_residual_error = (uf_result + error) % 2
    uf_flag = (surface_code.lz @ uf_residual_error % 2).any()
    if uf_flag == 0:
        uf_num_success += 1

    # Our decoder
    for j in range(n):

        bp_decoder.decode(syndrome)  # run BP for T times
        e_hard_values = bp_decoder.bp_decoding

        if (np.dot(H, e_hard_values) % 2 == syndrome).all():
            our_residual_error = (e_hard_values + error) % 2
            our_flag = (surface_code.lz @ our_residual_error % 2).any()
            if our_flag == 0:
                our_num_success += 1
                break
        else:
            # print(f"{bp_decoder.log_prob_ratios}")
            # print(f"max value = {max(abs(bp_decoder.log_prob_ratios))}")
            max_index = np.argmax(abs(bp_decoder.log_prob_ratios))
            if W[max_index] >= 0:
                W[max_index] = LFN
            else:
                W[max_index] = -LFN

        bp_decoder.update_channel_probs(W)  # TODO: 得传入概率，用w去更新channel probs


our_success_rate = our_num_success / num_trials
bposd_success_rate = bposd_num_success / num_trials
uf_success_rate = uf_num_success / num_trials
print(f"\nTotal trials: {num_trials}")
print(f"Our Success rate: {our_success_rate * 100:.2f}%")
print(f"BP+OSD Success rate: {bposd_success_rate * 100:.2f}%")
print(f"UF Success rate: {uf_success_rate * 100:.2f}%")


Total trials: 10000
Our Success rate: 50.94%
BP+OSD Success rate: 95.27%
UF Success rate: 86.28%


In [13]:
import numpy as np
iswap = np.array([[1,0,0,0],[0,0,1j,0],[0,1j,0,0],[0,0,0,1]])
swap = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])
iswap@iswap@swap

array([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j]])